# Dragon Utilities

This notebook contains all the utility functions required for diffusing the dragon. If you're looking for the 'final' Dragon class that wraps all the functionality, that'll be in the next notebook.

In [ ]:
#| default_exp utils

In [ ]:
# | export
from __future__ import annotations
import math, random, torch, matplotlib.pyplot as plt, numpy as np, matplotlib as mpl, shutil, os, gzip, pickle, re, copy
from pathlib import Path
from operator import itemgetter
from itertools import zip_longest
from functools import partial
import fastcore.all as fc
from glob import glob

from torch import tensor, nn, optim
import torch.nn.functional as F
from tqdm.auto import tqdm
import torchvision.transforms.functional as TF
from torch.optim import lr_scheduler
from diffusers import UNet2DModel
from torch.utils.data import DataLoader, default_collate
from torch.nn import init

from einops import rearrange
from fastprogress import progress_bar
from PIL import Image
from torchvision.io import read_image,ImageReadMode
from torchvision import transforms

from dataclasses import dataclass
from diffusers import LMSDiscreteScheduler, UNet2DConditionModel, AutoencoderKL, DDIMScheduler
from transformers import AutoTokenizer, CLIPTextModel
from diffusers.utils import BaseOutput

from diffusers.configuration_utils import ConfigMixin, register_to_config
from diffusers.utils import BaseOutput, deprecate
from diffusers.schedulers.scheduling_utils import SchedulerMixin

from dragon_diffusion.core import *
from dragon_diffusion.masks import *

### Feature hooks

These functions collect features from a specific layer in the model for each forward pass.

In [ ]:
#| export
def get_features(hook, layer, inp, out):
    if not hasattr(hook, 'feats'): hook.feats = out
    hook.feats = out

In [ ]:
#| export
class Hook():
    def __init__(self, model, func): self.hook = model.register_forward_hook(partial(func, self))
    def remove(self): self.hook.remove()
    def __del__(self): self.remove()

### Attention control

In [ ]:
#| export
from diffusers.models.attention_processor import AttnProcessor, Attention

The function below will take a custom attention processor, and inject into a specific section of the model. The paper discusses using attention control 'only in the decoder', and using the `location` parameter enables this level of dexterity.

In [ ]:
#| export
def get_attn_dict(processor, model, attention_type='all', location='all'):
    """
        Enables selective replacement of attention processors. For DragonDiffusion 
        use attention_type='attn1' (self-attention blocks) and location='decoder'. Looks
        overly complicated but just enables fine-grained control of where in the model
        the attention processors are replaced.
    """
    assert attention_type in ['all', 'attn1', 'attn2']
    assert location in ['all', 'decoder']
    attn_procs = {}
    for name in model.attn_processors.keys():
        attn_procs[name] = AttnProcessor()
    if attention_type == 'all' and location == 'all':
        for key in attn_procs.keys(): attn_procs[key] = processor(name=key)
    elif attention_type == 'attn1' and location == 'decoder':
        keys = [k for k in attn_procs.keys() if 'up_blocks' in k and 'attn1' in k]
        for key in keys: attn_procs[key] = processor(name=key)
    elif attention_type == 'attn1' and location == 'all':
        keys = [k for k in attn_procs.keys() if 'attn1' in k]
        for key in keys: attn_procs[key] = processor(name=key)
    elif attention_type == 'attn2' and location == 'decoder':
        keys = [k for k in attn_procs.keys() if 'up_blocks' in k and 'attn2' in k]
        for key in keys: attn_procs[key] = processor(name=key)
    elif attention_type == 'attn2' and location == 'all':
        keys = [k for k in attn_procs.keys() if 'attn2' in k]
        for key in keys: attn_procs[key] = processor(name=key)
    elif attention_type == 'all' and location == 'decoder':
        keys = [k for k in attn_procs.keys() if 'up_blocks' in k]
        for key in keys: attn_procs[key] = processor(name=key)
    return attn_procs

The function below is a simple storage object for attention. Specifically, it stores the `key` and `value` matrices from the forward pass of the diffusion branch concerned with reconstructing the original image, so that this can be injected into the editing branch.

In [ ]:
#| export
class AttnStorage:
    def __init__(self): self.storage = {}
    def __call__(self, name, key, value): 
        if not name in self.storage: self.storage[name] = {}
        self.storage[name]['key'] = key
        self.storage[name]['value'] = value
    def flush(self): self.storage = {}

Below is a custom attention processor. It doesn't do anything fancy, it just takes the `key` and `value` matrices from the forward passes and commits them to storage in the `attn_storage` object (above). It also enables injection of key and value matrices from the storage object. This is how we will do attention control.

In [ ]:
#| export
class CustomAttnProcessor(AttnProcessor):
    def __init__(self, attn_storage, name=None): 
        fc.store_attr()
        self.store_attention = False
        self.inject_attention = False
    def set_attention(self, store, inject): 
        self.store_attention = store
        self.inject_attention = inject
    def __call__(self, attn: Attention, hidden_states, encoder_hidden_states=None, attention_mask=None):
        batch_size, sequence_length, _ = (
            hidden_states.shape if encoder_hidden_states is None else encoder_hidden_states.shape
        )
        attention_mask = attn.prepare_attention_mask(attention_mask, sequence_length, batch_size)
        query = attn.to_q(hidden_states)

        if encoder_hidden_states is None:
            encoder_hidden_states = hidden_states
        elif attn.norm_cross:
            encoder_hidden_states = attn.norm_encoder_hidden_states(encoder_hidden_states)

        key = attn.to_k(encoder_hidden_states)
        value = attn.to_v(encoder_hidden_states)
     
        query = attn.head_to_batch_dim(query)
        key = attn.head_to_batch_dim(key)
        value = attn.head_to_batch_dim(value)
        
        if self.store_attention: self.attn_storage(self.name, key, value) ## stores the key and value matrices
        
        if self.inject_attention: ## enables injection of corresponding key and value matrices
            key = self.attn_storage.storage[self.name]['key']
            value = self.attn_storage.storage[self.name]['value']
        
        attention_probs = attn.get_attention_scores(query, key, attention_mask)
        attention_probs.requires_grad_(True)
        
        hidden_states = torch.bmm(attention_probs, value)
        hidden_states = attn.batch_to_head_dim(hidden_states)

        # linear proj
        hidden_states = attn.to_out[0](hidden_states)
        # dropout
        hidden_states = attn.to_out[1](hidden_states)
        
        return hidden_states

Finally, the function below just makes it easier to flick the switch for the attention processor between normal, storage and injection modes.

In [ ]:
#| export
def prepare_attention(model, attn_storage, set_store=False, set_inject=False):
    assert not (set_store is True and set_inject is True)
    for name, module in model.attn_processors.items(): 
        if "CustomAttnProcessor" in module.__class__.__name__: module.set_attention(set_store, set_inject)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()